<a href="https://colab.research.google.com/github/Lalitha-DS/Capstone/blob/main/Fine_Tuning_AI_Models_with_LoRA_and_Deploying_with_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install transformers accelerate peft datasets torch streamlit

In [23]:
!pip install streamlit

In [24]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType
import streamlit as st

In [26]:
# For this project, let's assume we're using a dataset containing supply chain risk-related data.
dataset = load_dataset("csv", data_files={"train": "/content/train_data_qa (1).csv", "test": "/content/test_data_qa (1).csv"})

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set the pad token to eos_token
model = GPT2LMHeadModel.from_pretrained("gpt2")

def tokenize_function(examples):
    return tokenizer(examples["question"], padding="max_length", truncation=True)

dataset = dataset.map(tokenize_function, batched=True)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [27]:
### Step 4: Apply LoRA Fine-Tuning

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["c_attn"],  # Changed target modules to 'c_attn'
    task_type="CAUSAL_LM"  # Add task type for causal language modeling
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.2364


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [28]:
model.save_pretrained("./fine_tuned_gpt2")
tokenizer.save_pretrained("./fine_tuned_gpt2")

('./fine_tuned_gpt2/tokenizer_config.json',
 './fine_tuned_gpt2/special_tokens_map.json',
 './fine_tuned_gpt2/vocab.json',
 './fine_tuned_gpt2/merges.txt',
 './fine_tuned_gpt2/added_tokens.json')

In [35]:
%%writefile app.py
import streamlit as st
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load fine-tuned model and tokenizer
model_path = "./fine_tuned_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# Load QA datasets
train_data = pd.read_csv("test_data_qa (1).csv")
test_data = pd.read_csv("test_data_qa (1).csv")

# Combine both datasets into a dictionary
qa_data = pd.concat([train_data, test_data], ignore_index=True)
qa_dict = dict(zip(qa_data["question"], qa_data["answer"]))

st.title("AI-Powered Supply Chain Q&A")
st.write("Ask a supply chain-related question and get an AI-generated response!")

# User input
user_question = st.text_area("Enter your question:")

def generate_gpt2_response(question):
    input_ids = tokenizer.encode(question, return_tensors="pt")
    output = model.generate(input_ids, max_length=100, num_return_sequences=1)
    return tokenizer.decode(output[0], skip_special_tokens=True)

if st.button("Get Answer"):
    if user_question in qa_dict:
        st.write("### AI Answer:")
        st.write(qa_dict[user_question])
    else:
        st.write("### AI-Generated Answer:")
        generated_answer = generate_gpt2_response(user_question)
        st.write(generated_answer)


Overwriting app.py


In [36]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from peft import PeftModel

# Load fine-tuned model and tokenizer
model_path = "./fine_tuned_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = PeftModel.from_pretrained(GPT2LMHeadModel.from_pretrained("gpt2"), model_path)

def generate_answer(question):
    input_ids = tokenizer(question, return_tensors="pt").input_ids
    output = model.generate(input_ids, max_length=100)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Load QA dataset for validation
import pandas as pd
qa_data = pd.read_csv("/content/train_data_qa (1).csv")
qa_data = pd.read_csv("/content/test_data_qa (1).csv")
qa_dict = dict(zip(qa_data["question"], qa_data["answer"]))

if __name__ == "__main__":
    user_input = input("Enter a supply chain-related question: ")

    if user_input in qa_dict:
        print("\nAI Answer:")
        print(qa_dict[user_input])
    else:
        print("\nNo match found. The input question is not in the dataset.")


Enter a supply chain-related question: What is the role of IoT in supply chain optimization?

AI Answer:
IoT enables real-time tracking of shipments, monitoring of goods, and proactive maintenance of supply chain equipment.


In [37]:
!curl https://loca.lt/mytunnelpassword

35.233.168.75

In [38]:

!streamlit run app.py &>/content/logs.txt &


In [39]:
!npx localtunnel --port 8501

⠙⠹⠸⠼⠴your url is: https://fluffy-files-taste.loca.lt
^C
